In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing the libraries required

In [3]:
import cv2
import os
import glob
import numpy as np
import random

Pre-processing function

In [ ]:
def prepare(input):
    # preprocessing the image input
    clean = cv2.fastNlMeansDenoising(input) #denoise the image
    ret, tresh = cv2.threshold(clean, 127, 1, cv2.THRESH_BINARY_INV)  #apply threshholding to pixels
    img = crop(tresh)   #crop the image

    # 40x10 image as a flatten array
    flatten_img = cv2.resize(img, (40, 10), interpolation=cv2.INTER_AREA).flatten()

    # resize to 400x100
    resized = cv2.resize(img, (400, 100), interpolation=cv2.INTER_AREA)
    columns = np.sum(resized, axis=0)  # sum of all columns
    lines = np.sum(resized, axis=1)  # sum of all lines

    h, w = img.shape
    aspect = w / h

    return [*flatten_img, *columns, *lines, aspect]


def crop(img):
    points = cv2.findNonZero(img)
    x, y, w, h = cv2.boundingRect(points)
    return img[y: y+h, x: x+w]

Neural Network and its functions

In [ ]:
class NeuralNetwork():

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1],sizes[1:])]

    def feedforward(self, a):
        for b,w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def separate_batches(self, training_data, batch_size):
        random.shuffle(training_data)
        n = len(training_data)
        return [training_data[i:i + batch_size] for i in range(0, n, batch_size)]

    def update_batches(self, batches, alpha):
        for batch in batches:
            nabla_b = [np.zeros(b.shape) for b in self.biases]
            nabla_w = [np.zeros(w.shape) for w in self.weights]
            m = len(batch)
            for x, y in batch:
                delta_b, delta_w = self.backpropagation(x, y)
                nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_b)]
                nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_w)]

            self.weights = [w - (alpha / m) * nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b - (alpha / m) * nb for b, nb in zip(self.biases, nabla_b)]

    def backpropagation(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            # layer-bound b and w
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def sgd(self, training_data, epochs, batch_size, alpha, test_data):
        n_test = len(test_data)
        error=0

        for epoch in range(epochs):
            batches = self.separate_batches(training_data, batch_size)
            self.update_batches(batches, alpha)
            genuine=self.evaluate(test_data)
            error = error+abs(genuine-12)
            print("Epoch {0}: {1} / {2}".format(epoch, genuine, n_test))
        error=error/epochs
        print("Result deviation: {0}%".format(error*100/12))

    def evaluate(self, test_data):
      for (x,y) in test_data:
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return output_activations - y


def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

Main function to train and test the model

In [ ]:
def main():
    print('OpenCV version {} '.format(cv2.__version__))
    training_data = []
    for filename in glob.iglob('/content/drive/My Drive/sign-recg/Data/training/024/*.png', recursive=True):
        img = cv2.imread(filename, 0)
        if img is not None:
            data = np.array(prepare(img))
            data = np.reshape(data, (901, 1))
            result = [[0], [1]] if "genuine" in filename else [[1], [0]]
            result = np.array(result)
            result = np.reshape(result, (2,1))
            training_data.append((data, result))
    test_data = []
    for filename in glob.iglob('/content/drive/My Drive/sign-recg/Data/testing/024/*.png', recursive=True):
        img = cv2.imread(filename, 0)
        if img is not None:
            data = np.array(prepare(img))
            data = np.reshape(data, (901, 1))
            result = 1 if "genuine" in filename else 0
            test_data.append((data, result))
            
    net = NeuralNetwork([901, 500, 500, 2])
    net.sgd(training_data, 4, 50, 0.01, test_data)

In [25]:
main()

OpenCV version 4.1.2 
Epoch 0 : 13 / 24
Epoch 1 : 13 / 24
Epoch 2 : 13 / 24
Epoch 3 : 13 / 24
Epoch 4 : 13 / 24
Epoch 5 : 13 / 24
Epoch 6 : 13 / 24
Epoch 7 : 13 / 24
Epoch 8 : 13 / 24
Epoch 9 : 13 / 24
Result deviation: 7.666666666666667%
